# Tutorial: LQR and Region of Atraction for the Simple Pendulum

In this tutorial, we will look at some optimal controllers for our simple pendulum. To do so, we will first linearize our system so that we can use a Linear-Quadratic Regulator. Then, we will use the PendulumPlant class to simulate it. We will also combine our LQR with a controller we have previously seen in order to provide a solution that stabilizes the system. Finally, we will test the controllers that we have developed on a real, remote, pendulum.

**Pre-requisites**

- Access to the Cloud Pendulum
- Familiarity with infinite-horizon Linear-Quadratic Regulators.
- Understanding of LaSalle's invariance principle and Lyapunov Analysis.

**Goals**

- Learning to implement controllers that guarantee stability under the correct conditions.
- Understanding and using the Region of Attraction concept to stabilize our system.
- Performing a swing-up with a real simple pendulum.

This notebook is organized as follows:

    1. Introduction
        1.1. The simple pendulum
        1.2. Energy Shaping
    2. Linear-Quadratic Regulators
        2.1. Linearization of the equations of motion
        2.2. Infinite-horizon LQR
    3. Region of Attraction
        3.1. LaSalle's invariance principle
        3.2. Energy Shaping + LQR

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import scipy.linalg

%matplotlib inline

from IPython.display import HTML

from pendulum_plant import PendulumPlant, plot_timeseries
from utils import direct_sphere
from plot import plot_ellipse

## 1. Introduction
### 1.1. The simple Pendulum

<center><img src="media/pendulum_undamped_axes.png" width="400"></center>
    
Here, we can take a look again at how the *PendulumPlant* works...

The cell below constructs a pendulum object with the specified dynamic parameters.

In [ ]:
mass = 0.06                    # mass at the end of the pendulum [kg]
length = 0.1                   # length of the rod [m]
damping = 0.0004               # viscious friction [kg m/s]
gravity = 9.81                 # gravity [kg m²/s]
inertia = mass*length*length   # inertia of the pendulum [kg m²]
torque_limit = np.inf          # torque limit of the motor, here we assume the motor can produce any torque

pendulum = PendulumPlant(mass=mass,
                         length=length,
                         damping=damping,
                         gravity=gravity,
                         inertia=inertia,
                         torque_limit=torque_limit)

In order to simulate and animate a motion of the pendulum you can call the simulate_and_animate class method. The arguments are:

    - t0: the initial time
    - y0: the initial state
    - tf: the final time
    - dt: the integration timestep
    - controller: controller object providing torques for the motor (introduced later)
    - integrator: the integration method. we will be using "runge_kutta"
    
The method returns the trajectory split in to time (T), state (X) and torques (U) as well as an animation object.

**Note:** If you do not need the animation you can also call the method *simulate* with the same parameters. The method returns T, X, U but no animation object. Making an animation **takes some time**.

In the next cell, we simulate the damped pendulum for 10 seconds:

In [ ]:
%%capture
T, X, U, anim = pendulum.simulate_and_animate(
              t0=0.0,
              y0=[2.0, 0.0],
              tf=10.0,
              dt=0.01,
              controller=None,
              integrator="runge_kutta")

In [ ]:
HTML(anim.to_html5_video())

You can use the plot_timeseries function to plot the trajectories (position, velocity and torque). Of course you can also plot these in any other way you like.

In [ ]:
plot_timeseries(T, X, U)

For testing/debugging it might be better to simply simulate without animation and then see the plots.

In [ ]:
Tsim_ff, Xsim_ff, Usim_ff = pendulum.simulate(
              t0=0.0,
              y0=[1.57, 0.0],
              tf=10.0,
              dt=0.01,
              controller=None,
              integrator="runge_kutta")
plot_timeseries(Tsim_ff, Xsim_ff, Usim_ff)

### 1.2. Energy Shaping

The Energy Shaping controller uses the total energy in the system to make it easier to reach a state which is energetically very far from our current one while respecting the limit on torque that can be provided. It works in the following way:

- First, we calculate the current total mechanical energy of our system. It is given by the following expression:

$$
E_{total} = E_{potential} + E_{kinetic}
$$
$$
E_{potential} = m g l (1 -\cos(\theta))
$$
$$
E_{kinetic} = \frac{1}{2} I \dot{\theta}^2
$$

- Then, we calculate the total energy associated with our goal state, $E_{goal}$
- Lastly, we define a simple control law that increases the energy of the system if it is below that of the goal state or decreases it otherwise.

$$
u = -K (E_{goal} - E_{total})
$$

where $u$ is the torque provided by the motor

In [ ]:
class EnergyShapingController():
    def __init__(self, mass, length, damping, torque_limit, gravity, K):
        self.K = K
        self.m = mass
        self.l = length
        self.g = gravity
        self.b = damping
        self.torque_limit = torque_limit

    def get_control_output(self, x):
        # Create a Energy Shaping Controller for swing up of simple pendulum 
        Ted = self.m * self.l * self.g  # desired energy
        Tec = (self.m * (self.l * x[1])**2)/2 - self.m * self.l * self.g * np.cos(x[0]) # current energy
        tau_es = -self.K * x[1] * (Tec - Ted) + self.b*x[1]
        tau = np.clip(tau_es,-self.torque_limit, self.torque_limit)
        
        return tau

In [ ]:
energy_shaping_controller = EnergyShapingController(mass=mass, length=length, damping = damping, torque_limit=0.03, gravity=gravity, K=0.1)

In [ ]:
%%capture
T, X, U, anim = pendulum.simulate_and_animate(
              t0=0.0,
              y0=[0.001, 0.0],
              tf=10.0,
              dt=0.01,
              controller=energy_shaping_controller,
              integrator="runge_kutta")

Remember that running the following cell may take a while.

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
plot_timeseries(T, X, U)

In [ ]:
Tsim_es, Xsim_es, Usim_es = pendulum.simulate(
              t0=0.0,
              y0=[0.001, 0.0],
              tf=10.0,
              dt=0.01,
              controller=energy_shaping_controller,
              integrator="runge_kutta")
plot_timeseries(Tsim_es, Xsim_es, Usim_es)


# Export the data to csv file
simulation_csv_data_es = np.array([Tsim_es, np.asarray(Xsim_es).T[0], np.asarray(Xsim_es).T[1],Usim_es]).T
np.savetxt("es_simulation_data_es.csv", simulation_csv_data_es, delimiter=',', header="time,pos,vel,tau", comments="")

## 2. Linear-Quadratic Regulators

We will use a LQR to hold the "up" position. That way, when we reach the top, we can remain in that position.

### 2.1 Linearization of the equations of motion

To be able to use a LQR, we must first write the equations governing our system in a way that fits the following expression:

$$
\dot{x} = Ax + Bu
$$

For the simple pendulum, this is the system dynamics, which currently have the shape:

$$
I \ddot{\theta} = \tau - m g l \sin(\theta) - b \dot{\theta}
$$

where $I = m l^2$

Consider our state vector $x$ to be $\left[ \begin{matrix} \theta - \theta^* & \dot{\theta} - \dot{\theta}^* \end{matrix} \right]^T$. We can use Taylor series to linearize the dynamics around a point. To linearize a function around a point $x^*$ with a Taylor expansion, the following equation is applied:

$$
f(x) \approx \sum_{n=0}^{\infty} f^{(n)}(x^*)\frac{(x-x^*)^n}{n!}
$$

### Think-Pair-Share (10 min)

- Using a first order Taylor series, linearize the dynamics around $\theta^* = \pi$, $\dot{\theta}^* = 0$.
- Rewrite the linearized equations of motion into the shape $\dot{x} = Ax+Bu$.
- Implement the linearized dynamics in the *LQRController* class.

In [ ]:
class LQRController():
    def __init__(self, mass, length, gravity, damping, torque_limit, Q, R):
        self.Q = Q                 # 2x2 Matrix cost on state error
        self.R = R                 # 1x1 Matrix cost on actuation
        self.m = mass
        self.l = length
        self.g = gravity
        self.b = damping
        self.torque_limit = torque_limit
        A, B = self.linearize()
        self.K = np.zeros((2, 2))
        # Continuous Time LQR
        self.K, self.S, _ = lqr(A, B, Q, R)
        print("LQR Gain Matrix (K):", self.K)
        # Discrete Time LQR
        # self.K, _, _ = dlqr(A, B, Q, R)
    
    def linearize(self):
        # Linearized equations of motion for simple pendulum
        # at the upright/topmost state (pi, 0)
        # Either Continuous form: x_dot = Ax + Bu
        # Or discrete form: x[n+1] = Ax[n] + Bu[n]

        ### 
        A = np.array([[0, 0], [0, 0]]) # Type here
        B = np.array([[0],[0]]) # Type here
        ###
        
        return A, B

    def get_control_output(self, x):
        # Use the LQR Gain matrix K to create the controller.
        tau = 0 # Type here
        # Add a torque limit before returning the tau value.
        return np.clip(tau[0], -self.torque_limit, self.torque_limit)

### 2.2. Infinite-horizon LQR

For our optimal infinite-horizon LQR controller, we need to define a cost function. A simple, convex, cost function is the following:

$$
J = \int_0^\infty \left[ \mathbf{x}^T \mathbf{Q} \mathbf{x} + \mathbf{u}^T \mathbf{R} \mathbf{u} \right] dt,
    \quad \mathbf{Q} = \mathbf{Q}^T \succeq \mathbf{0}, \mathbf{R} = \mathbf{R}^T \succ 0.
$$

Our goal with this controller is to find the cost-to-go function that solves the following Hamilton-Jacobi-Bellman equation:

$$
\forall \mathbf{x}, \quad 0 = \min_{\mathbf{u}} \left[ \mathbf{x}^T \mathbf{Q}
    \mathbf{x} + \mathbf{u}^T \mathbf{R} \mathbf{u} + \frac{\partial J^*}{\partial \mathbf{x}} \left( \mathbf{A} \mathbf{x} + \mathbf{B} \mathbf{u} \right)
    \right].
$$

Where $J^*(\mathbf{x})$ is the function that provides the cost that we will incurr to get from point $\mathbf{x}$ to the goal. For this problem, our cost-to-go function will have the following shape:

$$
J^*(\mathbf{x}) = \mathbf{x}^T \mathbf{S} \mathbf{x}, \quad \mathbf{S} = \mathbf{S}^T \succeq 0
$$

The gradient of the cost-to-go function then is:

$$ \frac{\partial J^*}{\partial \mathbf{x}} = 2 \mathbf{x}^T \mathbf{S}$$

With this information we can then derive the control law that will produce the optimal control policy:

$$
\mathbf{u}^* = -\mathbf{R}^{-1} \mathbf{B}^T \mathbf{S} \mathbf{x} = -\mathbf{K} \mathbf{x}
$$

After substituting $\mathbf{u}$ in the HJB equation with the optimal control policy, we get the following equation:

$$
0 = \mathbf{S} \mathbf{A} + \mathbf{A}^T \mathbf{S} - \mathbf{S} \mathbf{B} \mathbf{R}^{-1} \mathbf{B}^T \mathbf{S} + \mathbf{Q}
$$

This is the continuous Riccati equation, which can be solved using scipy's *solve_continuous_are*. This is implemented in the *lqr* function.

In [ ]:
def lqr(A,B,Q,R):
    """Solve the continuous time lqr controller.
    dx/dt = A x + B u
    cost = integral x.T*Q*x + u.T*R*u
    """
    #ref Bertsekas, p.151

    #Solve the Algebraic Riccati Equation
    S = scipy.linalg.solve_continuous_are(A, B, Q, R)

    #compute the LQR gain
    K = scipy.linalg.inv(R).dot(B.T.dot(S))
    eigVals, eigVecs = scipy.linalg.eig(A-B.dot(K))
    return K, S, eigVals

Now, we can define the weights of the cost function and instantiate the LQR controller.

### Think-Pair-Share (5 min)
- Tune the LQR controller by modifying the penalties associated to the position and velocity error. $\mathbf{Q}$ and $\mathbf{R}$ should take the shapes:

$$
\mathbf{Q} = \left[ \begin{matrix} w_{\theta} & 0 \\ 0 & w_{\dot{\theta}} \end{matrix} \right], \; \mathbf{R} = \left[ \begin{matrix} w_{u} \end{matrix} \right]
$$

In [ ]:
Q = np.array([[0, 0], [0, 0]]) # Type here
R = np.array([[0]]) # Type here
lqr_controller = LQRController(mass=mass, length=length, gravity=gravity, damping=damping, torque_limit=0.03, Q=Q, R=R)

Simulating the pendulum starting from a state close to the goal, we should see that the controller indeed stabilizes the pendulum in the up position.

In [ ]:
%%capture
T, X, U, anim = pendulum.simulate_and_animate(
              t0=0.0,
              y0=[np.pi-0.2, -0.1],
              tf=10.0,
              dt=0.01,
              controller=lqr_controller,
              integrator="runge_kutta")

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
plot_timeseries(T, X, U)

In [ ]:
Tsim_lqr, Xsim_lqr, Usim_lqr = pendulum.simulate(
              t0=0.0,
              y0=[np.pi-0.2, -0.1],
              tf=10.0,
              dt=0.01,
              controller=lqr_controller,
              integrator="runge_kutta")
plot_timeseries(Tsim_lqr, Xsim_lqr, Usim_lqr)

# Measured Position
plt.figure
plt.plot(Tsim_lqr, np.asarray(Xsim_lqr).T[0])
plt.xlabel("Time (s)")
plt.ylabel("Position (rad)")
plt.title("Position (rad) vs Time (s)")
plt.show()
# Measured Velocity
plt.figure
plt.plot(Tsim_lqr, np.asarray(Xsim_lqr).T[1])
plt.xlabel("Time (s)")
plt.ylabel("Velocity (rad/s)")
plt.title("Velocity (rad/s) vs Time (s)")
plt.show()
# Measured Torque
plt.figure
plt.plot(Tsim_lqr, Usim_lqr)
plt.xlabel("Time (s)")
plt.ylabel("Torque (Nm)")
plt.title("Torque (Nm) vs Time (s)")
plt.show()

# Export the data to csv file
simulation_csv_data_lqr = np.array([Tsim_lqr, np.asarray(Xsim_lqr).T[0], np.asarray(Xsim_lqr).T[1],Usim_lqr]).T
np.savetxt("es_simulation_data_lqr.csv", simulation_csv_data_lqr, delimiter=',', header="time,pos,vel,tau", comments="")

## 3. Region of Attraction

We can repeat this test for a set of randomly chosen starting points in the vicinity of the goal state and show that the controller will stabilize the system around the goal state.

In [ ]:
X_LQR = []
for i in range(10):
    # Your Code here
    # Generate random states here in the neighbourhood of (pi, 0)
    #y0_random = [random.uniform(np.pi-0.001,np.pi+0.001), random.uniform(-0.005,0.005)]
    y0_random = [random.uniform(np.pi-0.2,np.pi+0.2), random.uniform(-0.8,0.8)]
    #y0_random = [random.uniform(np.pi-0.5,np.pi+0.5), random.uniform(-0.5,0.5)]
    T, X, U = pendulum.simulate(
                t0=0.0,
                y0=y0_random,
                tf=10.0,
                dt=0.01,
                controller=lqr_controller,
                integrator="runge_kutta")
    X_LQR.append(X)

In [ ]:
# Plot Positions
for i in range(10):
    plt.plot(T, np.asarray(X_LQR[i]).T[0], label=str(i))

plt.legend(loc="best")
plt.show()

In [ ]:
# Plot Velocities
for i in range(10):
    plt.plot(T, np.asarray(X_LQR[i]).T[1], label=str(i))

plt.legend(loc="best")
plt.show()

### Think-Pair-Share (5 min)

- If the system didn't reach the goal from all the starting points in the previous cells, tune it further so it will.
- Tune the controller so that it can reach the top from a far less convenient position like the one in the simulation from the following cells. Does it reach the top? Why (or why not)?

### 3.1. LaSalle's invariance principle

LaSalle's invariance principle states that if, for a system with continuous dynamics defined by $\dot{x} = f(x)$, we can produce a scalar function $V(x)$ with continuous derivatives for which we have:

- $V(\mathbf{x})>0 \;\;\; \forall \mathbf{x} \in \mathcal{G}, \mathbf{x} \neq \mathbf{0}$ and $V(\mathbf{0}) = 0$
- $\dot{V}(\mathbf{x}) \leq 0 \;\;\; \forall \mathbf{x} \in \mathcal{G}$

then, the origin is locally asymptotically stable and set $\mathcal{G}$ is contained within its region of attraction.

We can exploit this information to narrow down a set which is entirely within the region of attraction of the origin $x = \left[ \begin{matrix} \pi & 0\end{matrix} \right]^T$. A simple way of doing so is to pick a shape for set $\mathcal{G}$ and scale it so that it is within the region of attraction. 

We will use an ellipse centered on the origin and defined by:

$$
V(\mathbf{x}) \leq \rho
$$

Remember that the cost-to-go can be selected as a candidate Lyapunov function i.e., $V(\mathbf{x}) = \mathbf{x}^T S \mathbf{x}$

To find the value of $\rho$ that provides the largest region of attraction guaranteeing stability, we can follow the procedure:

- Set an initial value of $\rho$ which we know is unrealistically large.
- Start sampling points from inside the ellipse. If at that point the conditions outlined in LaSalle's invariance principle hold, that point is part of the region of attraction. If LaSalle's principle of invariance does not hold for that point, we reduce the size of the ellipse until the point is no longer in it.
- We repeat the previous step until we converge to the largest ellipse which only contains points in the region of attraction.

Complete the following cells so that this strategy is implemented.

In [ ]:
def sample_from_ellipsoid(M,rho,r_i=0,r_o=1):
    """sample directly from the ellipsoid defined by xT M x.

    Parameters
    ----------
    M : np.array
        Matrix M such that xT M x leq rho defines the hyperellipsoid to sample from
    rho : float
        rho such that xT M x leq rho defines the hyperellipsoid to sample from
    r_i : int, optional
        inner radius, by default 0
    r_o : int, optional
        outer radius, by default 1

    Returns
    -------
    np.array
        random vector from within the hyperellipsoid
    """
    lamb,eigV=np.linalg.eigh(M/rho) 
    d=len(M)
    xy=direct_sphere(d,r_i=r_i,r_o=r_o) #sample from outer shells
    T=np.linalg.inv(np.dot(np.diag(np.sqrt(lamb)),eigV.T)) #transform sphere to ellipsoid (refer to e.g. boyd lectures on linear algebra)
    return np.dot(T,xy.T).T

In [ ]:
def najafi_based_sampling(
    plant, controller, n=10000, rho0=100, M=None, x_star=np.array([np.pi, 0])
):
    """Estimate the RoA for the closed loop dynamics using the method introduced in Najafi, E., Babuška, R. & Lopes, G.A.D. A fast sampling method for estimating the domain of attraction. Nonlinear Dyn 86, 823–834 (2016). https://doi.org/10.1007/s11071-016-2926-7

    Parameters
    ----------
    plant : simple_pendulum.model.pendulum_plant
        configured pendulum plant object
    controller : simple_pendulum.controllers.lqr.lqr_controller
        configured lqr controller object
    n : int, optional
        number of samples, by default 100000
    rho0 : int, optional
        initial estimate of rho, by default 10
    M : np.array, optional
        M, such that x_barT M x_bar is the Lyapunov fct. by default None, and controller.S is used
    x_star : np.array, optional
        nominal position (fixed point of the nonlinear dynamics)

    Returns
    -------
    rho : float
        estimated value of rho
    M : np.array
        M
    points: list containing all the points that were tested
    """

    rho = rho0

    points = []
    
    if M is None:
        M = np.array(controller.S)
    else:
        pass

    for i in range(n):
        # sample initial state from sublevel set
        # check if it fullfills Lyapunov conditions
        x_bar = sample_from_ellipsoid(M, rho)
        x = x_star + x_bar

        tau = controller.get_control_output([x[0], x[1]])

        xdot = plant.rhs(0, x, tau)

        V = 0 # Type here your candidate Lyapunov function, you may use @ in python for matmult

        Vdot = 2 * np.dot(x_bar, np.dot(M, xdot))

        if V > rho:
            print("something is fishy")
        # V < rho is true trivially, because we sample from the ellipsoid
        if Vdot > 0.0:  # if one of the lyapunov conditions is not satisfied
            rho = V

        points.append(x)
    
    return rho, M, points

In [ ]:
rho, M, points = najafi_based_sampling(pendulum, lqr_controller, n=1000)
print("rho: ", rho)

Run the following cell to plot the ellipse of the region of attraction and the points which have been sampled.

In [ ]:
plot_ellipse(np.pi, 0, rho, M, points)

### 3.2. Energy Shaping + LQR

We can now combine Energy Shaping and LQR to produce a controller which can achieve the swing-up with the underactuated simple pendulum and stabilize at the top.

### Think-Pair-Share 

The class defined in the following cell is a controller which combines an energy shaping controller with a LQR to stabilize itself once it has reached the top. Use $\mathcal{G}$ and/or $\rho$ to determine when the controller should be switched from Energy Shaping to LQR.

In [ ]:
class EnergyShapingAndLQRController():
    """
    Controller which swings up the pendulum with the energy shaping
    controller and stabilizes the pendulum with the lqr controller.
    """
    def __init__(self, mass=1.0, length=0.5, damping=0.1,
                 gravity=9.81, torque_limit=np.inf, K=1.0,
                 Q=np.diag((10, 1)), R=np.array([[1]]), compute_RoA=False):
        self.m = mass
        self.l = length
        self.b = damping
        self.g = gravity

        self.energy_shaping_controller = EnergyShapingController(mass=mass,
                                                                 length=length,
                                                                 damping=damping,
                                                                 gravity=gravity,
                                                                 torque_limit=torque_limit,
                                                                 K=K)
        self.lqr_controller = LQRController(mass=mass,
                                            length=length,
                                            damping=damping,
                                            gravity=gravity,
                                            torque_limit=torque_limit,
                                            Q=Q,
                                            R=R)

        self.active_controller = "none"
        self.swingup_time = None
        
        self.meas_time = 0.0

    def set_RoA(self, M, rho):
        self.M = M
        self.rho = rho

    def is_in_RoA(self, x):

        x = np.array(x)

        if (x.T @ self.M @ x) < self.rho:
            return True
        else:
            return False
    
    def get_control_output(self, x):
        # Measured State
        meas_pos = x[0] 
        meas_vel = x[1]
        # Desired Goal State
        des_pos = np.pi 
        des_vel = 0.0
        
        #u = self.lqr_controller.get_control_output(x)
        u = self.energy_shaping_controller.get_control_output(x)
        th = meas_pos + np.pi
        th = (th + np.pi) % (2*np.pi) - np.pi
        
        #if (np.abs(th) < self.eps[0] and
        #    np.abs(meas_vel) < self.eps[1]):
        
        if self.is_in_RoA([th, meas_vel]):
            #self.swingup_time = self.meas_time
            #print("Inside ROA!")
            #print("th:", th, "th_dot: ", meas_vel)
            if u is not None:
                if self.active_controller != "lqr":
                    self.active_controller = "lqr"
                    print("Switching to lqr")
                u = self.lqr_controller.get_control_output(x)
                #print("lqr: ", u)
                #print("u: ",u)
            else:
                if self.active_controller != "EnergyShaping":
                    self.active_controller = "EnergyShaping"
                    print("Switching to EnergyShaping")
                u = self.energy_shaping_controller.get_control_output(x)
                #print("ES: ", u)
        #else:
            #print("outside RoA")
        return u
    

In [ ]:
Q = np.array([[0.001, 0], [0, 0.0001]])
R = np.array([[1]])
K = 0.1
es_lqr_controller = EnergyShapingAndLQRController(mass=mass, length=length, damping=damping, gravity=gravity, torque_limit=0.03, K=K, Q=Q, R=R)
rho, M = najafi_based_sampling(pendulum, lqr_controller, n=1000)[0:2]
es_lqr_controller.set_RoA(M, rho)

In [ ]:
%%capture
T, X, U, anim = pendulum.simulate_and_animate(
              t0=0.0,
              y0=[0.001, 0.0],
              tf=10.0,
              dt=0.01,
              controller=es_lqr_controller,
              integrator="runge_kutta")

In [ ]:
HTML(anim.to_html5_video())

In [ ]:
plot_timeseries(T, X, U)

In [ ]:
Tsim_es_lqr, Xsim_es_lqr, Usim_es_lqr = pendulum.simulate(
              t0=0.0,
              y0=[0.001, 0.0],
              tf=5.0,
              dt=0.01,
              controller=es_lqr_controller,
              integrator="runge_kutta")
plot_timeseries(Tsim_es_lqr, Xsim_es_lqr, Usim_es_lqr)

# Measured Position
plt.figure
plt.plot(Tsim_es_lqr, np.asarray(Xsim_es_lqr).T[0])
plt.xlabel("Time (s)")
plt.ylabel("Position (rad)")
plt.title("Position (rad) vs Time (s)")
plt.show()
# Measured Velocity
plt.figure
plt.plot(Tsim_es_lqr, np.asarray(Xsim_es_lqr).T[1])
plt.xlabel("Time (s)")
plt.ylabel("Velocity (rad/s)")
plt.title("Velocity (rad/s) vs Time (s)")
plt.show()
# Measured Torque
plt.figure
plt.plot(Tsim_es_lqr, Usim_es_lqr)
plt.xlabel("Time (s)")
plt.ylabel("Torque (Nm)")
plt.title("Torque (Nm) vs Time (s)")
plt.show()

# Export the data to csv file
simulation_csv_data_es_lqr = np.array([Tsim_es_lqr, np.asarray(Xsim_es_lqr).T[0], np.asarray(Xsim_es_lqr).T[1],Usim_es_lqr]).T
np.savetxt("es_simulation_data_es_lqr.csv", simulation_csv_data_es_lqr, delimiter=',', header="time,pos,vel,tau", comments="")

## 4. Hardware Experiments with the Cloud Pendulum

The following image shows a hardware prototype of an active pendulum system actuated with a small direct drive actuator (GL35 from T-motors). The control electronics are provided by MAB robotics. This system has been integrated in Chalmers Cloud Pendulum infrastructure and you can remotely access the pendulum and get live experimental data from it with the comfort of your web-browsers. You get to be alpha testers of the system!

<div style="display: flex; justify-content: space-around;">
    <div><img src="media/pendulum_hardware.jpeg" width="400"></div>
    <div><img src="media/cp_fast.gif" width="450"></div>
</div>

In [ ]:
user_token = "Your User Token Here" 

## 4a. Energy Shaping Experiment

In [ ]:
tf = 5 # Final time (s)
dt = 0.01 # Time step (s)
Treal_es, Xreal_es, Ureal_es, Ureal_es_des, video_path = pendulum.run_on_hardware(tf, dt, controller=energy_shaping_controller, user_token=user_token, preparation_time = 5.0) 

# Measured Position
plt.figure
plt.plot(Treal_es, np.asarray(Xreal_es).T[0])
plt.xlabel("Time (s)")
plt.ylabel("Position (rad)")
plt.title("Position (rad) vs Time (s)")
plt.show()

# Measured Velocity
plt.figure
plt.plot(Treal_es, np.asarray(Xreal_es).T[1])
plt.xlabel("Time (s)")
plt.ylabel("Velocity (rad/s)")
plt.title("Velocity (rad/s) vs Time (s)")
plt.show()

# Measured Torque
plt.figure
plt.plot(Treal_es, Ureal_es)
plt.plot(Treal_es, Ureal_es_des)
plt.xlabel("Time (s)")
plt.ylabel("Torque (Nm)")
plt.title("Torque (Nm) vs Time (s)")
plt.show()

# Export the data to csv file
measured_csv_data_es = np.array([Treal_es, np.asarray(Xreal_es).T[0], np.asarray(Xreal_es).T[1],Ureal_es]).T
np.savetxt("es_measured_data_es.csv", measured_csv_data_es, delimiter=',', header="time,pos,vel,tau", comments="")

# Show the video
from IPython.display import Video
Video(video_path)

## 4b. LQR Experiment

In [ ]:
tf = 10 # Final time (s)
dt = 0.01 # Time step (s)
x0 = [2.7, 0.0] # Desired initial state. Note: desired velocity won't be respected in present implementation.
Treal_lqr, Xreal_lqr, Ureal_lqr, Ureal_lqr_des, video_path = pendulum.run_on_hardware(tf, dt, controller=lqr_controller, user_token=user_token, x0=x0, preparation_time = 5.0) 

# Measured Position
plt.figure
plt.plot(Treal_lqr, np.asarray(Xreal_lqr).T[0])
plt.xlabel("Time (s)")
plt.ylabel("Position (rad)")
plt.title("Position (rad) vs Time (s)")
plt.show()

# Measured Velocity
plt.figure
plt.plot(Treal_lqr, np.asarray(Xreal_lqr).T[1])
plt.xlabel("Time (s)")
plt.ylabel("Velocity (rad/s)")
plt.title("Velocity (rad/s) vs Time (s)")
plt.show()

# Measured Torque
plt.figure
plt.plot(Treal_lqr, Ureal_lqr)
plt.plot(Treal_lqr, Ureal_lqr_des)
plt.xlabel("Time (s)")
plt.ylabel("Torque (Nm)")
plt.title("Torque (Nm) vs Time (s)")
plt.show()

# Export the data to csv file
measured_csv_data_lqr = np.array([Treal_lqr, np.asarray(Xreal_lqr).T[0], np.asarray(Xreal_lqr).T[1],Ureal_lqr]).T
np.savetxt("es_measured_data_lqr.csv", measured_csv_data_lqr, delimiter=',', header="time,pos,vel,tau", comments="")

# Show the video
from IPython.display import Video
Video(video_path)

## 4c. ES+LQR Experiment

In [ ]:
tf = 5 # Final time (s)
dt = 0.01 # Time step (s)
Treal_es_lqr, Xreal_es_lqr, Ureal_es_lqr, Ureal_es_lqr_des, video_path = pendulum.run_on_hardware(tf, dt, controller=es_lqr_controller, user_token=user_token, preparation_time = 5.0) 

# Measured Position
plt.figure
plt.plot(Treal_es_lqr, np.asarray(Xreal_es_lqr).T[0])
plt.xlabel("Time (s)")
plt.ylabel("Position (rad)")
plt.title("Position (rad) vs Time (s)")
plt.show()

# Measured Velocity
plt.figure
plt.plot(Treal_es_lqr, np.asarray(Xreal_es_lqr).T[1])
plt.xlabel("Time (s)")
plt.ylabel("Velocity (rad/s)")
plt.title("Velocity (rad/s) vs Time (s)")
plt.show()

# Measured Torque
plt.figure
plt.plot(Treal_es_lqr, Ureal_es_lqr)
plt.plot(Treal_es_lqr, Ureal_es_lqr_des)
plt.xlabel("Time (s)")
plt.ylabel("Torque (Nm)")
plt.title("Torque (Nm) vs Time (s)")
plt.show()

# Export the data to csv file
measured_csv_data_es_lqr = np.array([Treal_es_lqr, np.asarray(Xreal_es_lqr).T[0], np.asarray(Xreal_es_lqr).T[1],Ureal_es_lqr]).T
np.savetxt("es_measured_data_es_lqr.csv", measured_csv_data_es_lqr, delimiter=',', header="time,pos,vel,tau", comments="")

# Show the video
from IPython.display import Video
Video(video_path)